<a href="https://colab.research.google.com/github/fabian-stettler/AD_Uebungen_HSLU/blob/master/01_systematic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Systematic Search

In the last session, we've prepared different classes that are useful to solve search problems. You've completed the `Node` class that we can use now for our first search strategy **breadth_first_graph_search**.

Compare your version with the one found on my [Github](https://github.com/iaherzog/search/blob/main/search.py) repository.

For the following exercises, we will clone the github repo and use the node class from the solution.






In [32]:
!git clone https://github.com/iaherzog/search.git
import sys
sys.path.append('/content/search')

fatal: destination path 'search' already exists and is not an empty directory.


## Breadth-first search

First, you are going to implement the breadth-first search strategy.

Hints:


- Create the queue for the frontier using `collections.py`. It implements high performance data types. The `collection.deque` allows you to easily extend the queue with `frontier.append` and to remove items from the queue with `frontier.popleft()`. An example of how to use this class is given below.
- The `breath_first_graph_search` function takes a `problem` as an argument and returns the goal node if it is found. The template for the function is given below.
- Remember that you can access the children of a node with the following code: `node.expand(problem)`.

In [33]:
from collections import deque

my_queue = deque()
my_queue.append('first_item')
my_queue.append('second_item')
my_queue.append('third_item')

print('get and remove first item')
print(my_queue.popleft())

print('get and remove first item now')
print(my_queue.popleft())

get and remove first item
first_item
get and remove first item now
second_item


Use the template below to implement the breadth first search algorithm.

In [34]:
from search import Node

def breadth_first_graph_search(problem):
    """ This implements the breadth first search strategy
        and returns the goal node """
    fifo_queue = deque()
    current_node = Node(problem.initial)
    fifo_queue.append(current_node)
    max_element_in_queue = 0
    flag = True

    while (problem.goal_test(current_node.state) == False):
      current_node = fifo_queue.popleft()
      #check for length of queue, exercise ILIAS
      if current_node.depth == 14 and flag:
        flag = False
        max_element_in_queue = len(fifo_queue)
      children = current_node.expand(problem)
      for child in children:
        fifo_queue.append(child)

    return current_node, max_element_in_queue







Lets create a map from the text book example to test if the algorithm is working.

In [35]:
from search import UndirectedGraph
romania_graph = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

With this, we can test our search algorithm. We define our search problem with the initial state Sibiu, the goal state Bucharest and the undirected graph romania_graph. Let's find the solution with the breadth-first search.

In [36]:
from search import GraphProblem
start = 'Sibiu'
goal = 'Bucharest'
problem = GraphProblem(start, goal, romania_graph)
goal_node, max_queue_elements = breadth_first_graph_search(problem)

The following code will show you some information about the solution and help you to troubleshoot your algorithm:

In [37]:
def evaluate(node):
    if node:
        print("The search algorithm reached " + node.state + " with a cost of " + str(node.path_cost) + ".")
        print("The actions that led to the solutions are the following: ")
        print(node.get_solution())
    else:
        print('no solution found')

evaluate(goal_node)

The search algorithm reached Bucharest with a cost of 310.
The actions that led to the solutions are the following: 
['Fagaras', 'Bucharest']


Compare the solution with the lecture slides (but note: here, Sibiu has more connections). If your solution is correct, you have successfully implemented your first search algorithm!

To evaluate the performance of your search algorithm, add the following features:

- print the depth of the solution found
- count how many nodes were visited
- print the maximum number of nodes that were stored at the same time



## Swiss Railway System ##

Let's try the algorithm on a larger data set. I've created a SBB class that can be used to import the data from the json file provided by the open data initiative of the swiss federal railways:



In [38]:
from sbb import SBB

sbb = SBB()
sbb.import_data('/content/search/linie-mit-betriebspunkten.json')

successfully imported 2787 hubs
successfully imported 401 train lines


The object `sbb` contains all the hubs and trainlines. For each hub, the x- and y-coordinates are given. To visualize the hubs, we can use the [folium](https://python-visualization.github.io/folium/modules.html) library.

In [40]:
import folium

map_ch = folium.Map(location=[46.8, 8.33],
                    zoom_start=8)

for hub in sbb.hubs:
    folium.CircleMarker(location=[sbb.hubs[hub].x, sbb.hubs[hub].y],
                        radius=2,
                        weight=4).add_to(map_ch)
map_ch


In this exercise, we are not restricted to the official train lines. If two hubs are connected, we can go from one hub to the other. If you have successfully implemented the classes above, the following code should execute and provide the directions between Rotkreuz and Thalwil.

In [ ]:
start = 'Rotkreuz'
goal = 'Thalwil'
sbb_graph = UndirectedGraph(sbb.create_map())
problem = GraphProblem(start, goal, sbb_graph)
goal_node, max_queue_elements = breadth_first_graph_search(problem)
print(max_queue_elements)

Let's print and visualize the solution.

In [ ]:
evaluate(goal_node)

def show_solution(map, goal_node):

    points = []

    for hub in goal_node.get_path_from_root():
        points.append([sbb.hubs[hub.state].x, sbb.hubs[hub.state].y])
        folium.CircleMarker(location=[sbb.hubs[hub.state].x, sbb.hubs[hub.state].y], color='red',
                        radius=2,
                        weight=4).add_to(map)
    folium.PolyLine(points, color='red').add_to(map)
    return map

m = show_solution(map_ch, goal_node)
m

##  More Uninformed Search Algorithms ##

As you know, the breadth-first search algorithm is just one of several systematic search strategies. Implement the following search algorithms and evaluate their performance. You might have to use the depth of the search tree.

1. Breadth-First Search (BFS) - already done :D
1. Depth-First Search (DFS)
1. Depth-Limited Search (DLS)
1. Iterative Deepening Search (IDS)


**TESTAT**: For the testat exercice on ILIAS, you only need to implement the first two algorithms.

Additional Questions:
- What is special about the sbb railway map in terms of complexity (branching factor, depth)?
- How could you preprocess the data set in order to reduce the search space?


In [ ]:
from search import Node

def depth_first_search(problem):
    """ This implements the breadth first search strategy
        and returns the goal node """
    lifo_queue = deque()
    current_node = Node(problem.initial)
    lifo_queue.append(current_node)
    max_element_in_queue = 0
    flag = True
    visited_nodes = []

    while (problem.goal_test(current_node.state) == False and lifo_queue):
      current_node = lifo_queue.pop()
      print(current_node.state)
      #check for length of queue, exercise ILIAS
      if current_node.depth == 15 and flag:
        flag = False
        max_element_in_queue = len(lifo_queue)
      if current_node not in visited_nodes:
        children = current_node.expand(problem)
        visited_nodes.append(current_node)
        for child in children:
          lifo_queue.append(child)
      if current_node in visited_nodes:


    return current_node, max_element_in_queue







In [ ]:
from search import UndirectedGraph
romania_graph = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

In [ ]:
from search import GraphProblem
start = 'Sibiu'
goal = 'Bucharest'
problem = GraphProblem(start, goal, romania_graph)
goal_node, max_queue_elements = depth_first_search(problem)

In [ ]:
def evaluate(node):
    if node:
        print("The search algorithm reached " + node.state + " with a cost of " + str(node.path_cost) + ".")
        print("The actions that led to the solutions are the following: ")
        print(node.get_solution())
    else:
        print('no solution found')

evaluate(goal_node)

In [ ]:
start = 'Rotkreuz'
goal = 'Thalwil'
sbb_graph = UndirectedGraph(sbb.create_map())
problem = GraphProblem(start, goal, sbb_graph)
goal_node, max_queue_elements = depth_first_search(problem)
print(max_queue_elements)